In [230]:
import pandas as pd
import json
import numpy as np
import re
import difflib as kwrd
pd.options.display.max_rows = 500

In [231]:
def similar(org,name):
    org = [y.replace('-',' ').replace('(',' ').replace(')',' ') for y in list(org['name'])]
    name = name.replace('-',' ').replace('(',' ').replace(')',' ')
    match = kwrd.get_close_matches(name,org,n=3, cutoff=0.5) 
    if len(match) > 0:
        return match[0]
    return None

In [232]:
with open ('../backend/dev/resources/files/organisations.json', 'r') as orgs:
    old = json.loads(orgs.read())

In [233]:
old = pd.DataFrame(old)
old['abbr'] = old['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
old['lname'] = old['name'].apply(lambda x: x.split('(')[0].lower())

In [234]:
new = pd.read_csv('./new_organisation.csv')
new['abbr'] = new['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
new['lname'] = new['name'].apply(lambda x: x.split('(')[0].lower())
new['review_status'] = "APPROVED"

In [235]:
old['match'] = old['name'].apply(lambda x: similar(new, x))

In [236]:
URL_REGEX = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
def getUrl(x):
    url = re.findall(URL_REGEX, x)
    if len(url) > 0:
        return "https://" + url[0][0].replace("https://","").replace("http://","")
    return None

In [237]:
merged = pd.merge(new, old, on="lname", how="outer", validate="one_to_one").replace([np.nan], [None])
merged['name'] = merged.apply(lambda x: x['name_x'] if x['name_x'] else x['name_y'], axis=1)
merged = merged[['name','country','country_group','type','url', 'program','contribution','expertise','review_status']]
merged['url'] = merged['url'].apply(lambda x: getUrl(x) if x else None)
merged = merged[['name','country','country_group','type','url', 'program','contribution','expertise','review_status']].sort_values('name')
merged['review_status'] = merged['review_status'].fillna("SUBMITTED")
merged['country_group'] = merged['country_group'].apply(lambda x: None if x == ' ' else x)

In [238]:
merged.to_csv('./result/organisation_merged.csv', index=False)

In [239]:
merged

,name,country,country_group,type,url,program,contribution,expertise,review_status
304,3M,None,None,None,None,None,None,None,SUBMITTED
1,5 Gyres Institute,United States of America,None,Non-governmental organization,https://www.5gyres.org/,None,None,None,APPROVED
305,Aberlpe,None,None,None,None,None,None,None,SUBMITTED
168,Adult commercial secondary School,Somalia,Africa,Academia and Research,None,None,None,None,APPROVED
306,Agence Française de Developpement (AFD),None,None,None,None,None,None,None,SUBMITTED
158,Agenda del Mar Comunicaciones,Colombia,Latin America and Carribean,Private Sector,https://www.agendadelmar.com,Clean up coastal\nSupport small business in po...,Education,Education,APPROVED
115,Alfred wegener Institute Helmholtz Centre for ...,Germany,Europe,Academia and Research,https://www.awi.de,member of national expert group on marine litt...,Reduced levels and impacts of (accumulated) ma...,Quantity and pathways of marine litter in the ...,APPROVED
53,All One Ocean,United States of America,None,Non-governmental organization,https://www.alloneocean.org/,None,None,None,APPROVED
295,Alliance to End Plastic Waste,Singapore,Asia and the Pacific,Non-governmental organization,https://endplasticwaste.org/,Prevent 3 Million metric tons of plastic waste...,global contribution to prevent plastic leakage,"Infrastructure, Innovation, Education & Engage...",APPROVED
307,Amazon,None,None,None,None,None,None,None,SUBMITTED


In [240]:
unlisted = merged[merged['type'].isna()]
unlisted.to_csv('./result/organisation_unlisted.csv')

In [241]:
merged.to_json('../backend/dev/resources/files/organisations_new.json', orient='records')

In [242]:
list(merged['country_group'].unique())

[None,
 'Africa',
 'Latin America and Carribean',
 'Europe',
 'Asia and the Pacific',
 'Global',
 'West Asia']